In [45]:
import pandas as pd
import numpy as np
import math
from funcoes import *
from tqdm import tqdm #biblioteca importada para estimar o tempo de compilação do código
pd.options.mode.chained_assignment = None
#dados foram baixados de https://www.cryptodatadownload.com/data/binance/ --> links para download: https://www.cryptodatadownload.com/cdd/Binance_BTCUSDT_minute.csv e https://www.cryptodatadownload.com/cdd/Binance_ETHUSDT_minute.csv
#baseado no paper: https://www.akademiabaru.com/doc/ARBMSV14_N1_P35_41.pdf


O objetivo do projeto é criar um modelo que seja capaz de realizar previsões sobre a variação do preço de um cryptoativo (bitcoin) em incrementos de tempo definidos, no caso, de minuto em minuto. Após extensa pesquisa, foi encontrado um artigo (link presente nos comentarios acima), que alegava ter conseguido uma boa acurácia utilizando uma regressão logística binomial. Portanto, partindo desse documento partimos para tentar implementar o modelo estátistico proposto pelo mesmo.

No documento acadêmico a técnica utilizada foi a de regressão logística binomial, de forma que a saída é 0 ou 1. 0 Indica que o valor do ativo irá cair no próximo incremento de tempo e 1 indica a subida no próximo incremento de tempo.

Começamos baixando um arquivo csv contendo colunas com valores de "open,high,low,close,Volume BTC,Volume USDT,tradecount" para cada minuto da criptomoeda bitcoin desde 2019 e calculamos os features e o target do modelo com esse dataframe.

Os features são indicadores técnicos, calculados para cada minuto (cada row do dataframe) e são funções de valores de rows anteriores do dataframe, para exemplificar, alguns são calculados com médias de preço dos últimos 12 incrementos de tempo, outros são calculados com o volume de trades no intervalo citado. Dos features escolhidos, todos são ou indicadores técnicos de "momentum" - que medem de diversas formas a taxa de aumento ou de queda do preço de um ativo ou são indicadores técnicos de "trend"- que tentam estimar tendências na variação do valor do ativo. O paper faz um ótimo trabalho de explicar a fundo cada indicador, em caso de dúvida.

Já o target, que é a variação do preço do ativo (positiva, ou negativa), é calculado pela subtração do preço de fechamento (close price) do ativo em dois tempos subsequentes, ou seja, sendo P(t) o close price do ativo, o target é igual ao "sinal" de P(t+1) - P(t), caso essa variação seja positiva o target é 1, caso seja negativa o target é 0.

Vale notar que alguns features são calculados com base em tempos anteriores, como por exemplo, o 12_SMA (Simple moving average de 12 incrementos de tempo) é obtido em função do close price de 12 incrementos de tempo anteriores e, portanto, para as primeiras 12 rows do dataframe esse feature será Nan. Assim sendo, é utilizado o df.dropna para eliminar as linhas nas quais alguns features ainda não estão definidos. Fora isso, não existem rows com valores nulos no df, deste modo, o uso do df.dropna não diminuirá o tamanho do dataframe de forma significativa e não afetará a acurácia do modelo. Ver imagem modelo.png 

Partindo para a lógica de treinamento e teste do modelo: foi definido um intervalo de 1000 incrementos de tempo de "lookback" para o modelo, isto é, toda regressão realizada teve seus coeficientes de regressão determinados à partir de 1000 incrementos de tempo anteriores (que equivalem à 1000 rows do dataframe). A título de exemplo, em um dataframe com 100000 dados, o modelo começa a operar na linha 1000. Ele determinará os coeficientes de regressão à partir de um fitting que utiliza os 1000 rows de features e targets anteriores e então, munido desses coeficientes ele insere os valores dos features da row em que se encontra (no caso a row 1000) e realiza uma estimativa. Ou seja, na primeira iteração (onde i = 1000) o modelo determina os coeficientes de regressão à partir das 1000 rows anteriores, e insere os valores da row 1000 na fórmula com os coeficientes de regressão encontrados e retorna a probabilidade do preço subir no próximo incremento de tempo, definimos uma linha média y = 0.5, de modo que, caso a probabilidade seja igual ou maior que 0.5 o output é igual 1 e caso a probabilidade seja menor que 0.5 o output é 0 (ver imagem grafico.png). A previsão é então inserida em uma lista que guarda os resultados produzidos pelo modelo para posteriormente ser comparada com os valores reais.

Importante notar que as previsões realizadas pelo modelo NÃO são usadas pelo mesmo na determinação de coeficientes de regressão em iterações futuras e, além disso, o modelo não tem acesso a "dados futuros" toda previsão realizada pelo modelo é feita exclusivamente em função das últimas 1000 rows de features (por meio da lista de listas de features) e deste modo, toda vez que o modelo realiza uma previsão é como se ele de fato estivesse no minuto em questão e tentasse prever a variação do preço do ativo para o próximo minuto.
  

Organizando dataframe de BTC para intervalos de 1 minuto

In [46]:

#número de rows escolhido foi de 100000, de forma a salvar tempo de processamento. Representa por volta de 69 dias. Chegamos a rodar o modelo com todas as linhas do dataframe, mas o tempo de compilação foi superior à 6 horas e, portanto, tornou-se inviável operar com um dataframe desse tamanho.
nrows = 100000
df = pd.read_csv("Binance_BTCUSDT_minute.csv",skiprows=1,nrows=nrows) 

calculate_indicators(df) #chamando função que foi definida no arquivo funcoes e que tem a finalidade de calcular os features em cada row e inseri-los no proprio dataframe original.
df = df.dropna(axis=0,how="any") #retirando as linhas com valores null ou nan do dataframe, lembrando que essas serão as linhas onde alguns features ainda não possuem dados antecedentes suficientes para serem calculados.
x,y = get_x_y(df) #chamando função que foi definida na file funcoes e que tem a finalidade de retornar duas listas, x - lista que guarda uma lista dos features para cada row e y - lista que guarda o change (target) 0 ou 1 para cada row.


In [47]:
interval = 1000 #definicao do intervalo (também chamado de "lookback"), isto é, o número de rows que o modelo usará para determinar os coeficientes de regressão em cada iteração.

falso_pos = 0
falso_neg = 0

lista_resultados = [] #iniciando variavel lista para guardar as previsões realizadas pelo modelo.
lista_coeficientes = [] #lista que guarda os coeficientes de regressao, não é utilizada no modelo mas serve para analisar a importância de cada um.

#loop que parte do intervalo, no caso row 1000 do df, e, com passo = 1, vai iterar por todo o dataframe realizando previsões do target seguindo a lógica já citada.
for i in tqdm(range(interval,len(x))):#tqdm gera a barra de estimativa do tempo para compilar o código.

    x_train,y_train = x[i-interval:i],y[i-interval:i] #selecionando o x e o y que vão ser usados no fit do modelo, para determinar os coeficientes de regressão da iteração. lembrando que estes serão sempre x[i-interval:i] e y[i-interval:i], ou seja, os ultimos 1000 valores de x e y sem contar a row atual - a row atual será usada como input para prever o target da iteração.
    x_test = x[i] # definindo os inputs que serão usados para calcular a probabilidade do preco da btc subir ou descer proximo minuto.
    
    coeficientes = calc_coefs(x_train,y_train) #chamando função que foi definida no arquivo funcoes; essa função recebe uma lista de listas (x_treino) e uma lista de targets e retorna os coeficientes de regressão.
    lista_coeficientes.append(coeficientes) #apendando os coeficientes de regressão à lista de coeficientes para poder avaliar a importância de cada feature posteriormente.

    lista_resultados.append(calcula_p(x_test,coeficientes)) #chamando função que foi definida no arquivo funcoes; essa função efetua a previsão da probabilidade do preço subir ou descer no minuto seguinte (target), para tal ela coloca os features da row em que se encontra (row=i) como inputs na função logística e os coeficientes da função logistica são os que foram calculados por meio do calc_coefs()
    
y = y[interval:] #retirando os primeiros 1000 elementos da lista de targets para poder comparar a lista das previsões que foram realizadas pelo modelo com a lista dos valores de fato.
x = x[interval:]

acertos_btc_min = 0 #número que guardará os acertos

linha_decisao = 0.5 #todas as probabilidades que o modelo definiu como acima de 0.5 são classificadas como 1 e abaixo de 0.5 como 0, lembrar da imagem da função logística.

for i in range(0,len(lista_resultados)): #loop que compara as previsões realizadas pelo modelo com os valores verdadeiros para determinar a acurácia do sistema.
    if lista_resultados[i] >= linha_decisao and y[i] == 1:
        acertos_btc_min += 1
    if lista_resultados[i] < linha_decisao and y[i] == 0:
        acertos_btc_min += 1
    if lista_resultados [i] >= linha_decisao and y[i] != 1:
        falso_pos += 1
    if lista_resultados [i] <= linha_decisao and y[i] != 0:
        falso_neg += 1
    
falso_pos = falso_pos/(len(y)-acertos_btc_min)
falso_neg = falso_neg/(len(y)-acertos_btc_min)

print("acurácia: "+str(acertos_btc_min/len(y)*100)) #dividindo número de acertos pelo tamanho da array de targets e multiplicando o resultado por 100 para determinar acurácia em %
print("dos erros {:.0%} são falsos positivos e {:.1%} são falsos negativos".format(falso_pos,falso_neg))


100%|██████████| 98975/98975 [27:04<00:00, 60.93it/s]

acurácia: 69.14473351856529
dos erros 51% são falsos positivos e 48.5% são falsos negativos


Percebe-se, finalmente, que a acurácia de 69.144% do sistema construído é muito próxima com a acurácia de 71.4% citada pelos acadêmicos e, portanto, o modelo pode ser considerado validado.

No paper são citadas duas acurácias: a maior, de 86%, usa "in sample data" - que não é o caso do nosso modelo. Já a menor, de 71.4%, é obtida usando somente "out of sample data" - que corresponde ao nosso caso.

Avaliando as importâncias médias de cada feature

In [48]:


#transpondo a lista de coeficientes de cada iteração (lista de listas)
trans = list(map(list, zip(*lista_coeficientes)))

colunas = list(df.columns)[9:]

importancias = []
#calculando a media de cada feature e a media dos coeficiente de regressao que multiplicavam cada feature
for i in range(0,len(colunas)):
    media_df = np.mean(df[colunas[i]])
    media_coef = np.mean(trans[i])
    importancias.append(media_df*media_coef)


import_df = pd.DataFrame(importancias,colunas)  
print("produto valor medio X coeficiente medio")
import_df


#coefs_import = [math.exp(np.mean(trans[0])*np.mean(df.MACD)),math.exp(np.mean(trans[1])*np.mean(df.RSI)),math.exp(np.mean(trans[2])*np.mean(df.SO)),math.exp(np.mean(trans[3])*np.mean(df["12_SMA"])),math.exp(np.mean(trans[5])*np.mean(df.ROC)),math.exp(np.mean(trans[6])*np.mean(df["tradecount"]))]

produto valor medio X coeficiente medio


,0
tradecount,-0.046889
MACD,0.035175
RSI,7.146344
SO,1.121305
12_SMA,1856.730989
12_EMA,-1865.272371
ROC,-0.000042


É importante notar, que no modelo de fato, todos esses produtos se tornam, na verdade, uma potência do número de Euler e, portanto, ao serem multiplicados na expressão de cálculo da probabilidade (ver figura modelo.png) todos acabam sendo somados e os valores dos features 12_SMA e 12_EMA que parecem absurdos isoladamente, tornam-se plausíveis visto que a soma de ambos (em média) é igual a aproximadamente -9, um valor mais próximo da faixa dos outros. Ademais percebe-se dessa análise que os features de maior importância são o RSI, o SO e a soma de 12_SMA e 12_EMA. 